load the pre-processed dataset

In [ ]:
df = pd.read_csv("df.csv")

In [ ]:
import openai
from bertopic.representation import OpenAI
from bertopic import BERTopic

# Create your representation model
client = openai.OpenAI(api_key="api_key")

In [ ]:
# System prompt describes information given to all conversations
system_prompt = """
You are a helpful, respectful and honest assistant for labeling topics.
"""

# Example prompt demonstrating the output we are looking for
example_prompt = """
I have a topic that contains the following documents:
- Traditional diets in most cultures were primarily plant-based with a little meat on top, but with the rise of industrial style meat production and factory farming, meat has become a staple food.
- Meat, but especially beef, is the word food in terms of emissions.
- Eating meat doesn't make you a bad person, not eating meat doesn't make you a good one.

The topic is described by the following keywords: 'meat, beef, eat, eating, emissions, steak, food, health, processed, chicken'. And a title 'The Evolution of Diets: From Plant-Based to Meat-Centric'.

Based on the information about the topic above, please create a short label of this topic, title, and texts related with this topic. Make sure you to only return the label, title, and texts related with this topic nothing more.

"""

main_prompt = """
I have a topic that contains the following documents:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short topic label and related texts in the following format:
topic: <topic label> text: <texts>
"""


prompt = system_prompt + main_prompt


In [ ]:
seed_words = ["Language","Dataset","Corpus Size","Computational","Machine Translation","Tokenization","Costs"
  "Interpretability", "Morphology","Semantic","Memory","Skewed distributions","Biased Distributions","Small size",
                "Generalizability","Bias","Hyperparameter","Hardware","Real world","Robustness", "Noisy", "Time",
                "Annotations", "Evaluation", "Diversity", "Segmentation", "Metrics"]

gpt-4-0125-preview

In [ ]:
# context window 128,000 tokens, returns maximum 4,096 tokens
from sklearn.metrics import silhouette_score
from bertopic import BERTopic
from umap import UMAP
import numpy as np
from hdbscan import HDBSCAN
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import openai
import tiktoken
from bertopic.representation import OpenAI
from bertopic import BERTopic
# from bertopic.representation import KeyBERTInspired

zeroshot_topic_list = seed_words
silhouette_score_list = []
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
# cluster_model = KMeans(31)
embeddings = sentence_model.encode(df_filtered_doc['Text'], show_progress_bar=False)
# Tokenizer
tokenizer= tiktoken.encoding_for_model("gpt-4-0125-preview")
# others: gpt-4-turbo-preview	, gpt-4, 	gpt-4-0613	, gpt-4-32k	, gpt-4-32k-0613

# umap_model = UMAP(n_neighbors=13, n_components=15,
#                     min_dist=0.0, metric='cosine',random_state=42) # If you want to reproduce the results, at the expense of performance, you can set a random_state in UMAP to prevent any stochastic behavior:

vectorizer_model = CountVectorizer(stop_words="english")

client = openai.OpenAI(api_key="api_key")

representation_model = OpenAI(
    client,
    model="gpt-4-0125-preview",
    delay_in_seconds=2,
    chat=True,
    nr_docs=4,
    doc_length=100,
    tokenizer=tokenizer,
    prompt = prompt
)

# Use the representation model in BERTopic on top of the default pipeline
# topic_model = BERTopic(representation_model=representation_model,umap_model=umap_model)

topic_model = BERTopic(
    embedding_model="thenlper/gte-small",
    min_topic_size=10,
    zeroshot_topic_list=zeroshot_topic_list,
    zeroshot_min_similarity=.75,
    representation_model=representation_model,
    vectorizer_model=vectorizer_model
)

topics, probs = topic_model.fit_transform(df_filtered_doc['Text'], embeddings)

# Generate `X` and `labels` only for non-outlier topics (as they are technically not clusters)
umap_embeddings = topic_model.umap_model.transform(embeddings)
indices = [index for index, topic in enumerate(topics) if topic != -1]
X = umap_embeddings[np.array(indices)]
labels = [topic for index, topic in enumerate(topics) if topic != -1]

  # Calculate silhouette score
silhouette_score(X, labels)


0.53972936

gpt turbo preview

In [ ]:
# context window 128,000 tokens, returns maximum 4,096 tokens
from sklearn.metrics import silhouette_score
from bertopic import BERTopic
from umap import UMAP
import numpy as np
from hdbscan import HDBSCAN
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import openai
import tiktoken
from bertopic.representation import OpenAI
from bertopic import BERTopic
# from bertopic.representation import KeyBERTInspired

zeroshot_topic_list = seed_words
silhouette_score_list = []
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
# cluster_model = KMeans(31)
embeddings = sentence_model.encode(df['Text'], show_progress_bar=False)
# Tokenizer
tokenizer= tiktoken.encoding_for_model("gpt-4-turbo-preview")
# others: gpt-4-turbo-preview	, gpt-4, 	gpt-4-0613	, gpt-4-32k	, gpt-4-32k-0613

# umap_model = UMAP(n_neighbors=13, n_components=15,
#                     min_dist=0.0, metric='cosine',random_state=42)

vectorizer_model = CountVectorizer(stop_words="english")

client = openai.OpenAI(api_key="api_key")

representation_model = OpenAI(
    client,
    model="gpt-4-turbo-preview",
    delay_in_seconds=2,
    chat=True,
    nr_docs=4,
    doc_length=100,
    tokenizer=tokenizer,
    prompt = prompt
)

# Use the representation model in BERTopic on top of the default pipeline
# topic_model = BERTopic(representation_model=representation_model,umap_model=umap_model)

topic_model = BERTopic(
    embedding_model="thenlper/gte-small",
    min_topic_size=10,
    zeroshot_topic_list=zeroshot_topic_list,
    zeroshot_min_similarity=.75,
    representation_model=representation_model,
    vectorizer_model=vectorizer_model,
    calculate_probabilities=True
)

topics, probs = topic_model.fit_transform(df['Text'], embeddings)

# Generate `X` and `labels` only for non-outlier topics (as they are technically not clusters)
umap_embeddings = topic_model.umap_model.transform(embeddings)
indices = [index for index, topic in enumerate(topics) if topic != -1]
X = umap_embeddings[np.array(indices)]
labels = [topic for index, topic in enumerate(topics) if topic != -1]

  # Calculate silhouette score
silhouette_score(X, labels)


0.58772725

In [ ]:
most_representative_words_llm = topic_model.get_topic_info()
most_representative_words_llm

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1018,-1_Challenges and Limitations in Language Mode...,[Challenges and Limitations in Language Model ...,[Despite our efforts to collect as many genera...
1,0,245,0_Challenges and Approaches in Multilingual La...,[Challenges and Approaches in Multilingual Lan...,[Even the premise of parsing questions to wiki...
2,1,92,1_Dialogue System Quality Assessment and Impro...,[Dialogue System Quality Assessment and Improv...,[The current dialogue system is mainly based o...
3,2,78,2_Multimodal Integration and Limitations in AI...,[Multimodal Integration and Limitations in AI ...,[Limitations of data collection our proposed d...
4,3,62,3_Reasoning in Large Language Models text: In ...,[Reasoning in Large Language Models text: In t...,[The proposed reasoning circuits framework int...
5,4,60,4_Advances and Challenges in Summarization Tas...,[Advances and Challenges in Summarization Task...,[We propose a set prediction network for the e...
6,5,51,5_Multi-hop QA Model Limitations and Challenge...,[Multi-hop QA Model Limitations and Challenges...,"[In this paper, we studied paragraph-level qag..."
7,6,51,6_Enhancing NER with Noisy Labels and Data Str...,[Enhancing NER with Noisy Labels and Data Stra...,[Safer framework is designed for handling bert...
8,7,46,7_Defense Strategies Against Adversarial Attac...,[Defense Strategies Against Adversarial Attack...,[Auxiliary attack and supports: votetrans with...
9,8,45,8_Challenges and Limitations in Clinical Data ...,[Challenges and Limitations in Clinical Data U...,[This study is based on a single clinical coho...


keyBERT

In [ ]:
# Convert each list in the 'Representation' column to a string
df['Representative_Docs'] = df['Representative_Docs'].apply(lambda x: ''.join(map(str, x)))


In [ ]:
from keybert import KeyBERT

kw_model = KeyBERT()
doc_embeddings, word_embeddings = kw_model.extract_embeddings(df["Representative_Docs"])

In [ ]:
keywords = kw_model.extract_keywords(df["Representative_Docs"], doc_embeddings=doc_embeddings, word_embeddings=word_embeddings)
keywords

[[('pretraining', 0.4151),
  ('pretrained', 0.3978),
  ('intratask', 0.3939),
  ('nlg', 0.3733),
  ('supervised', 0.3413)],
 [('multilinguality', 0.5652),
  ('multilingual', 0.5391),
  ('lingual', 0.454),
  ('languages', 0.4497),
  ('translations', 0.4419)],
 [('dialogue', 0.4775),
  ('conversations', 0.4157),
  ('conversation', 0.4124),
  ('evaluation', 0.358),
  ('chatbot', 0.322)],
 [('multimodal', 0.4893),
  ('pretrained', 0.343),
  ('attentionpruning', 0.3087),
  ('training', 0.3052),
  ('datasets', 0.2998)],
 [('reasoning', 0.4677),
  ('circuits', 0.4231),
  ('inference', 0.3629),
  ('logic', 0.3518),
  ('computational', 0.3454)],
 [('dialogue', 0.3993),
  ('summarization', 0.3801),
  ('dialog', 0.3376),
  ('graphbased', 0.3002),
  ('decoder', 0.2846)],
 [('qa', 0.392),
  ('sentences', 0.3907),
  ('qag', 0.3684),
  ('retrievals', 0.3428),
  ('languages', 0.3421)],
 [('labeling', 0.4199),
  ('labeled', 0.3923),
  ('classification', 0.3507),
  ('label', 0.3407),
  ('unlabeled', 0.3

In [ ]:
import nltk
import gensim
import gensim.corpora as corpora
import pandas as pd
import numpy as np
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from gensim.models.coherencemodel import CoherenceModel
import pandas as pd
import re

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_20newsgroups

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary


words_only = [[word for word, _ in sublist] for sublist in keywords]
print(words_only)
# convert list to string
df['Representative_Docs'] = df['Representative_Docs'].apply(lambda x: ''.join(x))
# Tokenize the strings to create a list of lists
list_of_tokens = df['Representative_Docs'].apply(lambda x: x.split()).tolist()

print(list_of_tokens)
word2id = Dictionary( list_of_tokens )

# Coherence model
cm = CoherenceModel(topics= words_only,
                    texts=list_of_tokens,
                    coherence='c_v',
                    dictionary=word2id)

coherence_per_topic = cm.get_coherence_per_topic()
print(coherence_per_topic)

import statistics
statistics.mean(coherence_per_topic)

[['pretraining', 'pretrained', 'intratask', 'nlg', 'supervised'], ['multilinguality', 'multilingual', 'lingual', 'languages', 'translations'], ['dialogue', 'conversations', 'conversation', 'evaluation', 'chatbot'], ['multimodal', 'pretrained', 'attentionpruning', 'training', 'datasets'], ['reasoning', 'circuits', 'inference', 'logic', 'computational'], ['dialogue', 'summarization', 'dialog', 'graphbased', 'decoder'], ['qa', 'sentences', 'qag', 'retrievals', 'languages'], ['labeling', 'labeled', 'classification', 'label', 'unlabeled'], ['adversarial', 'votetrans', 'attacks', 'overfitting', 'maliciously'], ['nlp', 'clinical', 'outcomes', 'prognostic', 'conversations'], ['rnns', 'rnn', 'encoder', 'benchmark', 'benchmarks'], ['conceptnet', 'knowledge', 'embedding', 'dialog', 'dialogue'], ['annotations', 'triggers', 'events', 'event', 'trigger'], ['contextual', 'lexical', 'embeddings', 'vocabulary', 'contexts'], ['tweets', 'nlp', 'linguistic', 'corpus', 'twitter'], ['nlp', 'corpus', 'citati

0.5136852081292358

# visualization

In [ ]:
topic_model.visualize_topics()


In [ ]:
topic_model.visualize_heatmap()


In [ ]:
from umap import UMAP

# Run the visualization with the original embeddings
topic_model.visualize_documents(df['Text'], embeddings=embeddings)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(df['Text'], reduced_embeddings=reduced_embeddings)

In [ ]:
from bertopic import BERTopic

# topic_model = BERTopic()
# topics, _ = topic_model.fit_transform(docs)
topic_distr, _ = topic_model.approximate_distribution(df['Text'], min_similarity=0)


In [ ]:
# don't use embedding_model = sentence model in 'BERTopic'
topic_model = BERTopic(representation_model=representation_model,vectorizer_model=vectorizer_model,
                         seed_topic_list = seed_words, verbose=True, calculate_probabilities=True)

topics, probs = topic_model.fit_transform(df['Text'], embeddings)

2024-03-19 08:06:30,148 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-19 08:06:39,094 - BERTopic - Dimensionality - Completed ✓
2024-03-19 08:06:39,095 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-19 08:06:39,366 - BERTopic - Cluster - Completed ✓
2024-03-19 08:06:39,372 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 33/33 [12:41<00:00, 23.07s/it]
2024-03-19 08:19:21,631 - BERTopic - Representation - Completed ✓


In [ ]:
# To visualize the probabilities of topic assignment
topic_model.visualize_distribution(probs[0])

# To visualize the topic distributions in a document
topic_model.visualize_distribution(topic_distr[0])

In [ ]:
topic_model.visualize_hierarchy()


In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(df['Text'])

topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)


100%|██████████| 31/31 [13:02<00:00, 25.25s/it]


In [ ]:
tree = topic_model.get_topic_tree(hierarchical_topics)
print(tree)

.
├─Limitations in NLP Model Development and Defense Methods text: - We foresee two limitations to our w
│    ├─Event Extraction and Representation in NLP text: The documents discuss advanced methods in Natural L
│    │    ├─Enhancing NLP Models with Syntactic and Semantic Structure Integration
text: .1, the generation perf
│    │    │    ├─■──Advances and Challenges in Parsing Techniques text: - This work follows in line with those studies w ── Topic: 23
│    │    │    └─Challenges and Limitations in Table Analysis and Metadata Extraction text: The documents discuss the
│    │    │         ├─■──Challenges and Limitations in Table Processing and SQL Query Generation text: Our synthetic pre-trai ── Topic: 24
│    │    │         └─■──Prompt-based Tuning and Optimization in Language Models text: First, the prompt templates are manual ── Topic: 27
│    │    └─Event Extraction and Representation Enhancements text: The documents discuss advancements in event e
│    │         ├─■──Information